<a href="https://colab.research.google.com/github/tyamamoto-1996/strabismus/blob/master/GradCAM%E3%81%AB%E3%82%88%E3%82%8B%E7%89%B9%E5%BE%B4%E5%8F%AF%E8%A6%96%E5%8C%96(Resnet18_50).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GradCam
参考： https://github.com/eclique/pytorch-gradcam

In [0]:
import argparse
import cv2
import numpy as np
import torch
from torch.autograd import Function
from torch import nn
from torchvision import models, transforms

import os
from matplotlib import pyplot as plt
from PIL import Image

import glob

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
'''
フォルダ構成
---dataset_all.zip-----train-----Cont
                   |          |--Exte
                   |          |--Inte
                   |
                   |---val-------Cont
                              |--Exte
                              |--Inte

My Drive---Deep_learning-----test_RN18/50.pth
                          |--synset_words.txt
'''

from google.colab import drive
drive.mount('/content/drive')

#dataset.zipを解凍

!date -R
!unzip -qq drive/My\ Drive/dataset_all.zip
!date -R
!ls

#パスの設定
PATH = '/content/drive/My Drive/Deep_learning/test_RN18.pth'

#model = models.vgg19(pretrained=True)
model = models.resnet18(pretrained=True)
#model = models.resnet50(pretrained=True)
#model = models.densenet201(pretrained=True)

In [0]:
# Opens image from disk, normalizes it and converts to tensor
read_tensor = transforms.Compose([
    lambda x: Image.open(x),
    lambda x: x.convert('RGB'),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
    lambda x: torch.unsqueeze(x, 0) #次元を1に引き延ばす
])

In [0]:
class Flatten(nn.Module): 
    """One layer module that flattens its input."""
    def __init__(self):
        super(Flatten, self).__init__()
    def forward(self, x):
        x = x.view(x.size(0), -1)
        return x

In [0]:
# Given label number returns class name
def get_class_name(c):
    labels = np.loadtxt('/content/drive/My Drive/Deep_learning/synset_words.txt', str, delimiter='\t')
    return ' '.join(labels[c].split(',')[0].split()[1:])

#Load model
Here, model is split into two parts: feature extractor and classifier. Provided is the implementation for ResNet/VGG/DenseNet architechtures.

Here, Flatten layer is being built in in the model as well. In PyTorch implementation flattenning is done in the forward pass, but we need it as a separate layer.

In [0]:
# モデルの設定
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device) #model_ftをGPUに載せる

# 重みロード
model.load_state_dict(torch.load(PATH))

# Split model in two parts
arch = model.__class__.__name__
if arch == 'ResNet':
    features_fn = nn.Sequential(*list(model.children())[:-2]) #最後の2層（AdaptiveAvgPool2dとLinear)を取り除いたもの
    classifier_fn = nn.Sequential(*(list(model.children())[-2:-1] + [Flatten()] + list(model.children())[-1:])) #最終層の前にFlatten()を挿入
elif arch == 'VGG':
    features_fn = nn.Sequential(*list(model.features.children())[:-1])
    classifier_fn = nn.Sequential(*(list(model.features.children())[-1:] + [Flatten()] + list(model.classifier.children())))
elif arch == 'DenseNet':
    features_fn = model.features
    classifier_fn = nn.Sequential(*([nn.AvgPool2d(7, 1), Flatten()] + [model.classifier]))

#評価モードにする    
model = model.eval()
model = model.cuda()

In [0]:
#ネットワークの確認
from torchsummary import summary

features_fn = features_fn.cuda()
summary(features_fn, (3,224,224))

In [0]:
print(classifier_fn)

#GradCam

In [0]:
def GradCAM(img, c, features_fn, classifier_fn):
    feats = features_fn(img.cuda())
    _, N, H, W = feats.size() #[1,2048,7,7]
    out = classifier_fn(feats) #out: [1,1000]
    c_score = out[0, c]   #c_scoreとは？？

    print(c)
    print(c_score)

    grads = torch.autograd.grad(c_score, feats)
    w = grads[0][0].mean(-1).mean(-1)           #ここでGlobalAveragePoolingをしている
    sal = torch.matmul(w, feats.view(N, H*W))
    sal = sal.view(H, W).cpu().detach().numpy()
    sal = np.maximum(sal, 0) #ReLUと同じ
    return sal

#1枚だけ実行

In [0]:
#画像のパスを指定
img_path = '/content/val/Exte/e1078.jpg'
img_tensor = read_tensor(img_path) #ここに前処理も含まれる
#Softmaxにかけたときの確率上位1つのpp(確率)とcc(class番号)を取得(tench→正常,goldfish→斜視)
pp, cc = torch.topk(nn.Softmax(dim=1)(model(img_tensor.to(device))), 1)

plt.figure(figsize=(15, 5))
#pとcを対にして入力
for i, (p, c) in enumerate(zip(pp[0], cc[0])):
    #グラフを1行3列に並べたうちのi番目
    plt.subplot(1, 3, i+1)
    sal = GradCAM(img_tensor, int(c), features_fn, classifier_fn)
    img = Image.open(img_path)
    #TensorをImageに変換
    sal = Image.fromarray(sal)
    sal = sal.resize(img.size, resample=Image.LINEAR)

    #plt.title('')
    plt.title('{}: {:.1f}%'.format(get_class_name(c), 100*float(p)))
    plt.axis('off')
    plt.imshow(img)
    plt.imshow(np.array(sal), alpha=0.5, cmap='jet')
plt.show()

#フォルダ内の画像全てに実行

In [0]:
#画像のパスを指定
img_dr = glob.glob('/content/val/*/*')

for img_path in img_dr:
    img_tensor = read_tensor(img_path) #ここに前処理も含まれる
    #Softmaxにかけたときの確率上位1つのpp(確率)とcc(class番号)を取得(tench→正常,goldfish→斜視)
    pp, cc = torch.topk(nn.Softmax(dim=1)(model(img_tensor.to(device))), 1)

    plt.figure(figsize=(15, 5))
    #pとcを対にして入力
    for i, (p, c) in enumerate(zip(pp[0], cc[0])):
        #グラフを1行3列に並べたうちのi番目
        plt.subplot(1, 3, i+1)
        sal = GradCAM(img_tensor, int(c), features_fn, classifier_fn)
        img = Image.open(img_path)
        #TensorをImageに変換
        sal = Image.fromarray(sal)
        sal = sal.resize(img.size, resample=Image.LINEAR)

        #plt.title('')
        plt.title('{}: {:.1f}%'.format(get_class_name(c), 100*float(p)))
        plt.axis('off')
        plt.imshow(img)
        plt.imshow(np.array(sal), alpha=0.5, cmap='jet')
    plt.show()